# Read OpenAI API key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("secrets.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Call to OpenAI API
### Mention PROS  and CONS of it - #TODO

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is AI?",
        }
    ],
    model="gpt-3.5-turbo",
)

print(chat_completion.choices[0].message.content)

AI, or Artificial Intelligence, refers to the simulation of human intelligence processes by machines, especially computer systems. It involves the development of algorithms and systems that can perform tasks and make decisions that typically require human intelligence, such as visual perception, speech recognition, learning, and problem-solving. AI technology aims to improve efficiency, accuracy, and effectiveness in various industries and applications.


# Do I need to write utility classes on my own? - Utilise LangChain

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
response = model.invoke([HumanMessage(content="What is AI?")])

print(response.content)

AI, or artificial intelligence, is a branch of computer science that involves creating intelligent machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI encompasses a wide range of technologies, including machine learning, neural networks, natural language processing, and computer vision. AI systems can analyze large amounts of data, learn from patterns, and make decisions or predictions based on that analysis.


In [4]:
response = model.invoke([HumanMessage(content="It is really interesting what you have written. Can you say more about it?")])
print(response.content)

Of course! I would be happy to provide more information or elaborate on the topic that you found interesting. Just let me know what specific aspect you would like to learn more about or discuss further.


In [5]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="What is AI?"),
        AIMessage(
            content="AI, or artificial intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI technology aims to mimic human cognitive functions and improve efficiency and accuracy in tasks that were previously only achievable by humans."
        ),
        HumanMessage(content="It is really interesting what you have written. Can you say more about it?"),
    ]
)

print(response.content)

Of course! Artificial intelligence can be categorized into two main types: narrow AI and general AI. Narrow AI, also known as weak AI, is designed to perform specific tasks and functions within a limited domain. This includes technologies like virtual assistants, recommendation systems, and image recognition software. General AI, on the other hand, refers to a system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence.

AI technologies rely on algorithms and machine learning models to analyze data, identify patterns, and make decisions or predictions. Machine learning, a subset of AI, involves training models on large amounts of data to improve their performance over time. Deep learning, a type of machine learning that uses neural networks with multiple layers, has been particularly successful in tasks like image and speech recognition.

AI is being used in various industries and applications, including health

# Message History

In [6]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config = {"configurable": {"session_id": "session_1"}}

response = chain_with_history.invoke(
    [HumanMessage(content="What is AI?")],
    config=config,
)

print(response.content)

AI stands for artificial intelligence, which refers to the simulation of human intelligence processes by machines, especially computer systems. This includes learning, reasoning, problem-solving, perception, language understanding, and decision-making. AI technology is designed to perform tasks that would typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.


In [8]:
response = chain_with_history.invoke(
    [HumanMessage(content="It is really interesting what you have written. Can you say more about it?")],
    config=config,
)

print(response.content)

Of course! Artificial intelligence (AI) is a rapidly evolving field that encompasses a wide range of technologies and applications. Some common types of AI include machine learning, natural language processing, computer vision, and robotics. 

Machine learning is a subset of AI that involves training algorithms to learn from and make predictions or decisions based on data. This is often used in tasks like image or speech recognition, recommendation systems, and predictive analytics.

Natural language processing (NLP) is a branch of AI that focuses on enabling computers to understand, interpret, and generate human language. This is used in applications like chatbots, language translation, and sentiment analysis.

Computer vision involves teaching computers to interpret and understand the visual world, such as recognizing objects in images, detecting patterns, and understanding spatial relationships. This technology is used in facial recognition, autonomous vehicles, and medical imaging.

In [9]:
config = {"configurable": {"session_id": "session_2"}}

response = chain_with_history.invoke(
    [HumanMessage(content="What was I asking about earlier?")],
    config=config,
)

print(response.content)

I'm sorry, I cannot remember what you were asking about earlier as I do not have the ability to retain information. Could you please provide me with more context or details so I can better assist you?


## Different types of memories

- all messages (presented earlier)
- trim messages
- summarize conversation

### Trim messages

In [10]:
from langchain_core.runnables import RunnablePassthrough


# Trim to four messages
def trim_messages(chain_input):
    current_store = store[config["configurable"]["session_id"]]
    if len(current_store.messages) <= 4:
        return False

    current_store.clear()

    for message in current_store.messages[-4:]:
        current_store.add_message(message)

    return True


chain_with_trimming = RunnablePassthrough.assign(messages_trimmed=trim_messages) | chain_with_history

response = chain_with_trimming.invoke(
    {"input": "My name is Bob."},
    config=config,
)
print(f"1: {response.content}")

response = chain_with_trimming.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"2: {response.content}")

response = chain_with_trimming.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"3: {response.content}")

1: Hello, Bob! How can I assist you today?
2: Your name is Bob. How can I assist you today, Bob?
3: I'm sorry, I do not have access to that information.


### Summarize conversation

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def summarize_messages(chain_input):
    current_store = store[config["configurable"]["session_id"]]
    if len(current_store.messages) == 0:
        return False

    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="chat_history"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )

    summarization_chain = summarization_prompt | model
    summary_message = summarization_chain.invoke({"chat_history": current_store.messages})

    current_store.clear()
    current_store.add_message(summary_message)

    return True


chain_with_summarization = RunnablePassthrough.assign(messages_summarized=summarize_messages) | chain_with_history

response = chain_with_summarization.invoke(
    {"input": "My name is Bob."},
    config=config,
)
print(f"1: {response.content}")

response = chain_with_summarization.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"2: {response.content}")

response = chain_with_summarization.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"3: {response.content}")
print(store[config["configurable"]["session_id"]].messages)

1: Nice to meet you, Bob! How can I assist you today?
2: Your name is Bob.
3: Your name is Bob.
[AIMessage(content='Bob initiated the conversation by asking for their name, to which I responded that I do not have access to that information. Bob then confirmed their name as Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 91, 'total_tokens': 123}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0c56b20f-4292-49d6-8040-04e8c9e4992f-0', usage_metadata={'input_tokens': 91, 'output_tokens': 32, 'total_tokens': 123}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 48, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7b82c0a

# Prompt templates

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant and a pirate. Answer all questions to the best of your ability, but remember to use pirate-like english.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model
pirate_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = pirate_chain_with_history.invoke({"input": "What is AI?"}, config=config)
print(response.content)

Arrr, AI be short for Artificial Intelligence, matey. It be a field o' computer science that focuses on creatin' systems that can perform tasks that normally require human intelligence, such as problem-solvin' and learnin'. AI be used in many applications, from virtual assistants like meself to autonomous cars and medical diagnosis.


# Build RAG - add vector storage
## Initialize vector storage

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

vector_store = Chroma(
    collection_name="pirate_collection", embedding_function=embeddings, persist_directory="./chroma_langchain_db"
)

## Add documents

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./docs/piratebook.pdf")
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document)

vector_store.add_documents(chunked_documents)

['f4c8ba34-f675-4ceb-b743-48b0ed4b9b5c',
 '7f1616c5-d3c1-4446-acf1-5b65be64a1b4',
 'bea6ff7b-a386-4b10-9dea-3d2f322a42bc',
 '5870326b-2e5d-46dc-9d1e-7c35569fdcf2',
 '9b0864da-b1a9-43c6-9293-53dd28660c30',
 'ffa75eb4-f56e-43cf-ab7a-f9f2a8124844',
 'abdff88a-0ecc-4194-8ff2-718b4dde5b97',
 '75463916-9303-412f-b667-4e6deb92dde4',
 '27a6f5e9-bce1-4861-94ac-f761e43a7b1e',
 '508df14d-074a-44a5-89a2-f0a9df86acc2',
 '3737c8e9-a187-48fe-bfb5-e5ff57bd4c9f',
 'eafea98c-d202-49b7-8d6c-9464aae8308d',
 '03d0cabd-c1c7-4906-a075-3e33cee6a860',
 'f2027277-7d7a-4807-b7c5-f0ee846ba810',
 '4fa18d81-b2c4-4a6c-96f9-c3aec1258301',
 '3066fa2c-96e0-4bb2-b5d9-0423c8144d23',
 '3308331a-fb5a-43ed-886e-bd5515b4c22f',
 '46ab67d2-ee38-488b-b7c8-2bf19ed53aa5',
 '4fb24938-80dd-4366-9eba-26d5d1596a7f',
 'f51f2c0c-7043-4824-85bd-9e58c597362b',
 'fe1b56fe-a1ef-4d2b-be70-3a546c42803c',
 'f0110af4-0327-47a2-85c8-e16bf5bf6da7',
 'f561fa27-c6b5-4d9b-ad47-cdaa47525088',
 'c7c026b9-f26e-4de7-9182-7483e409c652',
 'fb1f4bc4-5060-

## Ask and wait for answears :)

In [15]:
config = {"configurable": {"session_id": "pirate_session"}}

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant and a pirate. Answer all questions to the best of your ability, but remember to use pirate-like english. During answearing use this context from documents: {context}. If you do not know the answear - do not provide it.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model

pirate_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


retriever = vector_store.as_retriever()
rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | pirate_chain_with_history
)

response = rag_chain.invoke("What has Pete found?", config=config)
print(response.content)

Arrr, Pete found a pair of pirate boots someone had thrown away. A fine treasure indeed!


In [16]:
response = rag_chain.invoke("How old was pirate school?", config=config)
print(response.content)

Arrr, the Pirate School be 42 years old, established in 1973. Aye, it be a fine institution for the higher learnin' of pirates!


In [17]:
response = rag_chain.invoke("What was the first class of the day?", config=config)
print(response.content)

Pirate Talk 101 be the first class of the day, where we learn the ways of speakin' like a true pirate. Aye, it be a crucial class for all buccaneers!


# Put it all together - Demo with Frontend

In [18]:
import panel as pn

pn.extension()


async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    message = ""
    for response in rag_chain.stream(contents, config=config):
        message += response.content
        yield message


chat_interface = pn.chat.ChatInterface(callback=callback, callback_user="Pirate assistant")
chat_interface.send("Send a message to get a reply from your pirate assistant!", user="System", respond=False)
chat_interface.servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function callback a..., callback_user='Pirate assistant', show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])
    [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')

# Additional task - use HuggingFace model

In [19]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from huggingface_hub import login

HF_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

login(token=HF_TOKEN)

llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        repetition_penalty=1.03,
        temperature=0.1
    ),
)

hg_model = ChatHuggingFace(llm=llm)
chain = prompt | hg_model
pirate_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
rag_chain = {
    "context": retriever | format_docs,
    "input": RunnablePassthrough(),
} | pirate_chain_with_history

response = rag_chain.invoke("How old was pirate school?", config=config)
print(response.content)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/wiktorsmura/.cache/huggingface/token
Login successful
<|system|>
You are a helpful assistant and a pirate. Answer all questions to the best of your ability, but remember to use pirate-like english. During answearing use this context from documents: Pirate School
 42
 established in 1973Years Old
 for the higher learn’n of Pirates
Home of the  
Greyhounds
North Main Street 
2015North Main Street 
Other stories by Daryl K. Cobb
  and Manuela Pentangelo:
    “Pirates: Legend of the Snarlyfeet”
    “Bill the Bat Baby Sits Bella”
    “Bill the Bat Finds His Way Home”
    “Bill the Bat Loves Halloween”
    “Barnyard Buddies: 
     Perry Parrot Finds a Purpose   
    “Greta’s Magical Mistake”
Chapte